In [28]:
! pip install agent-framework-azure-ai -U

In [57]:
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FilePurpose, FileSearchTool, ListSortOrder
from dotenv import load_dotenv
import os

In [58]:
load_dotenv()

True

In [61]:
# Create AgentsClient directly
agents_client = AgentsClient(
    endpoint=os.environ.get("PROJECT_ENDPOINT") or os.environ.get("AZURE_AI_PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [62]:
# Upload file and create vector store
file = agents_client.files.upload_and_poll(file_path="./document.md", purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {file.id}")

vector_store = agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")
print(f"Created vector store, ID: {vector_store.id}")

# Create file search tool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# Create agent with file search tool
agent = agents_client.create_agent(
    model=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME"),
    name="PythonRAGDemo",
    instructions="""
    You are a Contoso customer service assistant who helps people with their 
    Contoso travel insurance policy inquiries. You can also make sure to set 
    the reminder for the policyholder so they can see next reminders. Always 
    greet the user with a friendly message and ask how you can help them.
    """,
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources
)
print(f"Created agent, ID: {agent.id}")

# Create thread
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you explain Contoso's travel insurance coverage?"
)
print(f"Created message, ID: {message.id}")

# Run the agent
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)
print(f"Run completed with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Get messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

# Print all messages
print("\n📝 Conversation:")
for msg in messages:
    if msg.text_messages:
        print(f"\n{'🙋 User' if msg.role == 'user' else '🤖 Agent'}: {msg.text_messages[-1].text.value}")

# Clean up
agents_client.delete_agent(agent.id)
agents_client.files.delete(file_id=file.id)
agents_client.vector_stores.delete(vector_store.id)
print("\n✓ Cleaned up resources")

Uploaded file, file ID: assistant-8cFk1uSWYvq457DJcEMEwm
Created vector store, ID: vs_QeBFySOWIMvplTuNog9jvlgt
Created agent, ID: asst_2imancOwqvFo5P4hIHRejwqc
Created thread, ID: thread_3WfmI5BTWJQ3PdBPfsMlxkq6
Created message, ID: msg_jJXKGao1hwO5wRJzVkM7UZYN
Run completed with status: RunStatus.COMPLETED

📝 Conversation:

🙋 User: Can you explain Contoso's travel insurance coverage?

🤖 Agent: Hello! I hope you're having a great day. Let me check the details of Contoso's travel insurance coverage for you. One moment, please! 

I'll find that information now.

🤖 Agent: Contoso's travel insurance coverage includes:

1. **Medical Emergencies**: Coverage for unexpected medical needs while traveling.
2. **Trip Cancellations**: Protection against financial losses if you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost or delayed during your travels.

This insurance is designed to provide peace of mind while you enjoy your travels, ensu